In [1]:
pip install torch_geometric

  Obtaining dependency information for torch_geometric from https://files.pythonhosted.org/packages/65/4e/6f9a75548a93fedcd4514ae2de9bee1e91bade6b73252b4da32f0e42ac52/torch_geometric-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch_geometric 
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.utils import to_dense_batch

from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('/kaggle/input/inn-competition/Trainset-Evaluation/Train_Set.csv')
ts = pd.read_csv('/kaggle/input/inn-competition/Transaction/InnoTech_Trans.csv')
# For prototype phase we remove the nan values
ts=ts.dropna()
ts = ts.reset_index().drop(columns='index')

# TEMP LINE CODE
df = df.drop_duplicates(subset=['CARD'])

ts = ts.drop(columns=['Terminal_ID'])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
le = LabelEncoder()  
le.fit(np.unique(np.concatenate([ts['Primary_ID'].values,ts['Second_ID'].values,df['CARD'].values])).tolist())

ts['Primary_ID'] = le.transform(ts['Primary_ID'])
ts['Second_ID'] = le.transform(ts['Second_ID'])
df['CARD'] = le.transform(df['CARD'])

# ts = pd.get_dummies(ts, columns=['Date'], prefix='Date')

In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

encoder = OneHotEncoder(sparse=False, drop='first')  # 'first' to drop one of the columns to avoid multicollinearity

column_transformer = ColumnTransformer(
    transformers=[('onehot', encoder, ['Date'])],
    remainder='passthrough'  # keep the other columns as they are
)

df_encoded = pd.DataFrame(column_transformer.fit_transform(ts))


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
ts=pd.concat([ts,df_encoded],axis=1).drop(columns=['Date'])

In [6]:
import torch
from torch_geometric.data import Data

def construct_graph(card_id,y):
    """
    This function will create a graph of transaction for each card base on the transaction dataset. The created graph
    is directed which means each CARD_ID can be source or destination of the transaction.

    params:
        card_id 
        ts: transaction dataset

    return:

        graph 
    """

    card_to_index = ts[ts['Primary_ID']==card_id].index
    card_in_index = ts[ts['Second_ID']==card_id].index
    
    edge_index_from = []
    edge_index_to = []
    
    node_features = [np.ones(104)]
    
    for i,index in zip(card_to_index,range(1,len(card_to_index)+1)):
        node_features.append(ts.loc[i].values)
        edge_index_from.append(0)
        edge_index_to.append(index)

    for i,index in zip(card_in_index,range(len(card_to_index)+1, 
                                           len(card_to_index)+len(card_in_index)+2+1)):
        node_features.append(ts.loc[i].values)
        edge_index_from.append(index)
        edge_index_to.append(0)

    data = Data(x=torch.tensor(node_features), edge_index=torch.tensor([edge_index_from,edge_index_to]),y=y)

    return data


In [7]:
from tqdm import tqdm
from multiprocessing import Pool

def process_data(args):
    i, y = args
    return construct_graph(i, y)

data = []

# Number of processes to use (adjust as needed)
num_processes = 4

# Create a pool of processes
with Pool(num_processes) as pool:
    # Use tqdm with imap_unordered to show progress
    for result in tqdm(pool.imap_unordered(process_data, zip(df['CARD'], df['LABEL']), chunksize=1), total=len(df)):
        data.append(result)


  0%|          | 0/3235 [00:00<?, ?it/s]/tmp/ipykernel_42/287827871.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  data = Data(x=torch.tensor(node_features), edge_index=torch.tensor([edge_index_from,edge_index_to]),y=y)
/tmp/ipykernel_42/287827871.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  data = Data(x=torch.tensor(node_features), edge_index=torch.tensor([edge_index_from,edge_index_to]),y=y)
/tmp/ipykernel_42/287827871.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Pleas

In [8]:
# Save the prepared data
torch.save(data,'/kaggle/working/graph_data.pt')

In [201]:
# import os
# import torch
# from torch_geometric.data import Dataset, Data
# from torch_geometric.utils import to_dense_batch

# class CustomGraphDataset(Dataset):
#     def __init__(self, root, transform=None):
#         self.root = root
#         self.transform = transform
#         self.graph_files =root
#         self.data = torch.load(self.graph_files)
#         self.max_num_nodes = self.find_max_num_nodes()
#         self.max_num_edge_index = self.find_max_num_edge_index()
        
#     def find_max_num_edge_index(self):
#         max_num_edge_index = max(data.edge_index.shape[1] for data in self.data)
#         return max_num_edge_index
    
#     def find_max_num_nodes(self):
#         max_num_nodes = max(data.num_nodes for data in self.data)
#         return max_num_nodes
    
#     def indices(self):
#         return list(range(self.len()))

#     def len(self):
#         return len(self.data)

#     def get(self, idx):
        
#         num_nodes_to_pad = self.max_num_nodes - self.data[idx].num_nodes
#         num_edge_to_pad = self.max_num_edge_index - self.data[idx].edge_index.shape[1]
        
#         # Pad num_nodes with zeros
#         self.data[idx].x = torch.cat([self.data[idx].x, torch.zeros(num_nodes_to_pad)])
        
#         # Pad edge_index with zeros
#         self.data[idx].edge_index = torch.cat([self.data[idx].edge_index, torch.zeros(2,num_edge_to_pad)],dim=1)
#         self.data[idx].edge_attr = torch.cat([self.data[idx].edge_attr,torch.zeros(num_edge_to_pad,104)])
#         # Update the number of nodes in the data object
#         self.data[idx].num_nodes = self.max_num_nodes

#         return self.data[idx]

In [202]:
# import torch
# from torch_geometric.data import Data
# from torch_geometric.utils import to_dense_batch

# def collate_fn(batch):
#     batch = to_dense_batch(batch)
#     return Data(x=batch.x, edge_index=batch.edge_index, y=batch.y)

In [203]:
# root = '/kaggle/working/graph_data.pt'
# train_dataset = CustomGraphDataset(root)
# train_dataset.max_num_nodes

24651

In [9]:
from torch_geometric.loader import DataLoader

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# # test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# # for step, data in enumerate(train_loader):
# #     print(f'Step {step + 1}:')
# #     print('=======')
# #     print(f'Number of graphs in the current batch: {data.num_graphs}')
# #     print(data)
# #     print()

In [10]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(data, batch_size=64, shuffle=True)

In [11]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)


In [16]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import BatchNorm1d  # Corrected import

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(104, hidden_channels)
        self.bn1 = BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.bn3 = BatchNorm1d(hidden_channels)
        self.lin = Linear(hidden_channels, 3)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = self.bn3(x)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x



model = GCN(64)

model.apply(weights_init)
print(model)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
#          print(data.x.shape)
#          print(data.edge_index.shape)
#          print(data.edge_attr.shape)
#          print(data.x.shape)
         data.cuda()
         out = model(data.x.float(), data.edge_index.int(), data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y.long())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()
     correct = 0
     for data in loader: # Iterate in batches over the training/test dataset.
         data.cuda()
         out = model(data.x.float(), data.edge_index.int(), data.batch)  # Perform a single forward pass.
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y.long()).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


GCN(
  (conv1): GCNConv(104, 64)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(64, 64)
  (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GCNConv(64, 64)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin): Linear(in_features=64, out_features=3, bias=True)
)


In [17]:
for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')


Epoch: 001, Train Acc: 0.6343
Epoch: 002, Train Acc: 0.5153
Epoch: 003, Train Acc: 0.2974
Epoch: 004, Train Acc: 0.6352
Epoch: 005, Train Acc: 0.4192
Epoch: 006, Train Acc: 0.5666
Epoch: 007, Train Acc: 0.6921
Epoch: 008, Train Acc: 0.4485
Epoch: 009, Train Acc: 0.6705
Epoch: 010, Train Acc: 0.6788
Epoch: 011, Train Acc: 0.6454
Epoch: 012, Train Acc: 0.6556
Epoch: 013, Train Acc: 0.6665
Epoch: 014, Train Acc: 0.6798
Epoch: 015, Train Acc: 0.6247
Epoch: 016, Train Acc: 0.6594
Epoch: 017, Train Acc: 0.6782
Epoch: 018, Train Acc: 0.6779
Epoch: 019, Train Acc: 0.6192
Epoch: 020, Train Acc: 0.6798
Epoch: 021, Train Acc: 0.6538
Epoch: 022, Train Acc: 0.6764
Epoch: 023, Train Acc: 0.4578
Epoch: 024, Train Acc: 0.7221
Epoch: 025, Train Acc: 0.7221
Epoch: 026, Train Acc: 0.6655
Epoch: 027, Train Acc: 0.5372
Epoch: 028, Train Acc: 0.7592
Epoch: 029, Train Acc: 0.7017
Epoch: 030, Train Acc: 0.7178
Epoch: 031, Train Acc: 0.6807
Epoch: 032, Train Acc: 0.6896
Epoch: 033, Train Acc: 0.6396
Epoch: 034

In [80]:
22137*104

2302248

In [19]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import BatchNorm1d  # Corrected import

import torch
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d
from torch_geometric.nn import global_mean_pool, GATConv

class GATSkip(torch.nn.Module):
    def __init__(self, hidden_channels, heads=1):
        super(GATSkip, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(104, hidden_channels, heads=heads)
        self.bn1 = BatchNorm1d(hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=heads)
        self.bn2 = BatchNorm1d(hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels, heads=heads)
        self.bn3 = BatchNorm1d(hidden_channels)
        self.lin = Linear(hidden_channels, 3)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings with skip connections
        x1 = self.conv1(x, edge_index)
        x1 = self.bn1(x1)
        x1 = F.relu(x1)

        x2 = self.conv2(x1, edge_index)
        x2 = self.bn2(x2)
        x2 = F.relu(x2)

        x3 = self.conv3(x2, edge_index)
        x3 = self.bn3(x3)

        # Skip connections
        x = x1 + x2 + x3

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x



model = GCN(256)

model.apply(weights_init)
print(model)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
#          print(data.x.shape)
#          print(data.edge_index.shape)
#          print(data.edge_attr.shape)
#          print(data.x.shape)
         data.cuda()
         out = model(data.x.float(), data.edge_index.int(), data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y.long())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()
     correct = 0
     for data in loader: # Iterate in batches over the training/test dataset.
         data.cuda()
         out = model(data.x.float(), data.edge_index.int(), data.batch)  # Perform a single forward pass.
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y.long()).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


GCN(
  (conv1): GCNConv(104, 256)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(256, 256)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GCNConv(256, 256)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin): Linear(in_features=256, out_features=3, bias=True)
)


In [20]:
for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')


Epoch: 001, Train Acc: 0.5471
Epoch: 002, Train Acc: 0.5839
Epoch: 003, Train Acc: 0.2856
Epoch: 004, Train Acc: 0.2983
Epoch: 005, Train Acc: 0.5376
Epoch: 006, Train Acc: 0.4862
Epoch: 007, Train Acc: 0.6269
Epoch: 008, Train Acc: 0.6986
Epoch: 009, Train Acc: 0.6646
Epoch: 010, Train Acc: 0.6862
Epoch: 011, Train Acc: 0.6958
Epoch: 012, Train Acc: 0.4662
Epoch: 013, Train Acc: 0.6575
Epoch: 014, Train Acc: 0.6612
Epoch: 015, Train Acc: 0.6386
Epoch: 016, Train Acc: 0.7076
Epoch: 017, Train Acc: 0.6479
Epoch: 018, Train Acc: 0.7113
Epoch: 019, Train Acc: 0.6563
Epoch: 020, Train Acc: 0.6921
Epoch: 021, Train Acc: 0.5051
Epoch: 022, Train Acc: 0.7008
Epoch: 023, Train Acc: 0.7131
Epoch: 024, Train Acc: 0.6971
Epoch: 025, Train Acc: 0.7147
Epoch: 026, Train Acc: 0.5750
Epoch: 027, Train Acc: 0.6989
Epoch: 028, Train Acc: 0.6903
Epoch: 029, Train Acc: 0.7199
Epoch: 030, Train Acc: 0.6383
Epoch: 031, Train Acc: 0.6825
Epoch: 032, Train Acc: 0.7264
Epoch: 033, Train Acc: 0.7422
Epoch: 034

In [21]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch.nn import BatchNorm1d  # Corrected import

import torch
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d
from torch_geometric.nn import global_mean_pool, GATConv

class GATSkip(torch.nn.Module):
    def __init__(self, hidden_channels, heads=1):
        super(GATSkip, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(104, hidden_channels, heads=heads)
        self.bn1 = BatchNorm1d(hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=heads)
        self.bn2 = BatchNorm1d(hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels, heads=heads)
        self.bn3 = BatchNorm1d(hidden_channels)
        self.lin = Linear(hidden_channels, 3)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings with skip connections
        x1 = self.conv1(x, edge_index)
        x1 = self.bn1(x1)
        x1 = F.elu(x1)

        x2 = self.conv2(x1, edge_index)
        x2 = self.bn2(x2)
        x2 = F.elu(x2)

        x3 = self.conv3(x2, edge_index)
        x3 = self.bn3(x3)

        # Skip connections
        x = x1 + x2 + x3

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x



model = GCN(256)

model.apply(weights_init)
print(model)
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
#          print(data.x.shape)
#          print(data.edge_index.shape)
#          print(data.edge_attr.shape)
#          print(data.x.shape)
         data.cuda()
         out = model(data.x.float(), data.edge_index.int(), data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y.long())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()
     correct = 0
     for data in loader: # Iterate in batches over the training/test dataset.
         data.cuda()
         out = model(data.x.float(), data.edge_index.int(), data.batch)  # Perform a single forward pass.
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y.long()).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


GCN(
  (conv1): GCNConv(104, 256)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(256, 256)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GCNConv(256, 256)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin): Linear(in_features=256, out_features=3, bias=True)
)


In [ ]:
for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')


Epoch: 001, Train Acc: 0.5481
Epoch: 002, Train Acc: 0.5861
Epoch: 003, Train Acc: 0.3184
Epoch: 004, Train Acc: 0.3054
Epoch: 005, Train Acc: 0.3376
Epoch: 006, Train Acc: 0.4436
Epoch: 007, Train Acc: 0.6272
